In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
%cd /content/drive/My Drive/SA/Code/spoof_detection_deep_features

/content/drive/My Drive/SA/Code/spoof_detection_deep_features


In [0]:
import keras 
import numpy as np
import librosa as lb
import pandas as pd
import os
import sys
# import pandas as pd
from keras.utils import to_categorical
# import soundfile as sf
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import History 
from keras.utils import plot_model,to_categorical
from keras.optimizers import SGD
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MinMaxScaler
# import speechpy as sp
# import statistics
from keras import backend as K
from keras.layers import Dense, Activation, Flatten

In [0]:
filename = "/content/drive/My Drive/SA/Code/spoof_detection_deep_features/data/ASVspoof2019/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt"

# open the file for reading
filehandle = open(filename, 'r')
train_protocol = []
while True:
    # read a single line
    line = (filehandle.readline())
    train_protocol.append(line)
    if not line:
        break

# close the pointer to that file
filehandle.close()

In [0]:
train_protocol = [s[:-1] for s in train_protocol]

In [0]:
train_protocol = pd.DataFrame([s.split(' ') for s in train_protocol])

In [0]:
train_protocol.head()

,0,1,2,3,4
0,LA_0079,LA_T_1138215,-,-,bonafide
1,LA_0079,LA_T_1271820,-,-,bonafide
2,LA_0079,LA_T_1272637,-,-,bonafide
3,LA_0079,LA_T_1276960,-,-,bonafide
4,LA_0079,LA_T_1341447,-,-,bonafide


In [0]:
train_protocol.columns = ['speaker_id','file_id', 'blah','system_id', 'label']

In [0]:
train_protocol = train_protocol[['speaker_id', 'file_id', 'system_id', 'label']]

In [0]:
train_protocol

,speaker_id,file_id,system_id,label
0,LA_0079,LA_T_1138215,-,bonafide
1,LA_0079,LA_T_1271820,-,bonafide
2,LA_0079,LA_T_1272637,-,bonafide
3,LA_0079,LA_T_1276960,-,bonafide
4,LA_0079,LA_T_1341447,-,bonafide
...,...,...,...,...
25376,LA_0098,LA_T_9779814,A06,spoof
25377,LA_0098,LA_T_9783312,A06,spoof
25378,LA_0098,LA_T_9839348,A06,spoof
25379,LA_0098,LA_T_9982036,A06,spoof


In [0]:
train_protocol = train_protocol.dropna()

In [46]:
train_protocol.iloc[1,1]

'LA_T_1271820'

In [44]:
#import names of files in dataset
path = r'/content/drive/My Drive/SA/Code/spoof_detection_deep_features/data/ASVspoof2019/LA/ASVspoof2019_LA_train/flac'
files = []
missing=[]
print(path)
for r, d, f in os.walk(path):
    for file in f:
        if '.flac' in file  :        
            files.append(os.path.join(r, file))
        else:
            missing.append(file)
print(len(files))


/content/drive/My Drive/SA/Code/spoof_detection_deep_features/data/ASVspoof2019/LA/ASVspoof2019_LA_train/flac
25400


In [0]:
files = [s.split('/') for s in files]

In [0]:
files = [s[-1] for s in files]

In [0]:
files = [s[:-5] for s in files]

In [0]:
files1 = [s.split(' ') for s in files]

In [0]:
for s in files:
  if len(s)>12:
    print(s)
    files.remove(s)

In [0]:
train_file_id = list(train_protocol.iloc[:,1])

In [79]:
train_file_id

['LA_T_1138215',
 'LA_T_1271820',
 'LA_T_1272637',
 'LA_T_1276960',
 'LA_T_1341447',
 'LA_T_1363611',
 'LA_T_1596451',
 'LA_T_1608170',
 'LA_T_1684951',
 'LA_T_1699801',
 'LA_T_1703395',
 'LA_T_1736342',
 'LA_T_1779188',
 'LA_T_1786825',
 'LA_T_1787246',
 'LA_T_1905558',
 'LA_T_2205687',
 'LA_T_2220901',
 'LA_T_2361751',
 'LA_T_2373806',
 'LA_T_2417641',
 'LA_T_2520083',
 'LA_T_2562689',
 'LA_T_2564579',
 'LA_T_2732709',
 'LA_T_2759900',
 'LA_T_2838981',
 'LA_T_2873890',
 'LA_T_2938316',
 'LA_T_3131990',
 'LA_T_3141223',
 'LA_T_3167734',
 'LA_T_3187715',
 'LA_T_3215578',
 'LA_T_3247389',
 'LA_T_3346004',
 'LA_T_3441957',
 'LA_T_3561136',
 'LA_T_3576775',
 'LA_T_3580508',
 'LA_T_3635733',
 'LA_T_3636245',
 'LA_T_3810183',
 'LA_T_3849267',
 'LA_T_3995089',
 'LA_T_3999087',
 'LA_T_3999267',
 'LA_T_4053835',
 'LA_T_4118798',
 'LA_T_4155454',
 'LA_T_4179989',
 'LA_T_4197307',
 'LA_T_4237787',
 'LA_T_4239283',
 'LA_T_4244329',
 'LA_T_4339311',
 'LA_T_4362109',
 'LA_T_4428691',
 'LA_T_4752788

In [0]:
train_labels = []
train_audio = []
for count,audio in enumerate(files):
  index = train_file_id.index(audio)
  if bool(index) == True:
    train_audio.append(lb.load('/content/drive/My Drive/SA/Code/spoof_detection_deep_features/data/ASVspoof2019/LA/ASVspoof2019_LA_train/flac/'+audio+'.flac',sr=16000))
    train_labels.append(train_protocol.iloc[index,3])
  if count%100 == 0 :
    print(count)
np.save("",np.array(train_audio))
np.save("",train_labels)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [83]:
index = train_file_id.index('LA_T_4941896')
print(train_protocol.iloc[index])

speaker_id         LA_0087
file_id       LA_T_4941896
system_id              A01
label                spoof
Name: 4170, dtype: object


In [101]:
train_audio

[(array([-0.00210571, -0.00198364, -0.00189209, ...,  0.00027466,
          0.00030518,  0.00033569], dtype=float32), 16000)]

In [106]:
train_labels

['bonafide']

In [0]:
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button").click() 
}setInterval(ClickConnect,60000)